In [1]:
import sys, os
sys.path.append(os.path.abspath(".."))

In [2]:
import cgael
from cgael.models.SingleColor import *

import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layer

import pygad
import pygad.kerasga

import numpy as np
import pandas as pd

In [3]:
ts = cgael.LanguageTokenSet("CHAT", '-')
gen = SimpleColorGenerator([Swatch.WHITE, Swatch.BLACK], blur=0, batch_lock=True)
model = SimpleColorModel(token_set=ts, word_count=2, word_length=3)

In [4]:
model.speaker.summary()

Model: "speaker"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1, 3)]            0         
                                                                 
 flatten (Flatten)           (None, 3)                 0         
                                                                 
 dense (Dense)               (None, 30)                120       
                                                                 
 reshape (Reshape)           (None, 2, 3, 5)           0         
                                                                 
Total params: 120 (480.00 Byte)
Trainable params: 120 (480.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [5]:
model.listener.summary()

Model: "listener"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 2, 3)]            0         
                                                                 
 embedding (Embedding)       (None, 2, 3, 4)           20        
                                                                 
 flatten_1 (Flatten)         (None, 24)                0         
                                                                 
 dense_1 (Dense)             (None, 3)                 75        
                                                                 
 reshape_1 (Reshape)         (None, 1, 3)              0         
                                                                 
Total params: 95 (380.00 Byte)
Trainable params: 95 (380.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [6]:
model.model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1, 3)]            0         
                                                                 
 speaker (Functional)        (None, 2, 3, 5)           120       
                                                                 
 argmax_layer (ArgmaxLayer)  (None, 2, 3)              0         
                                                                 
 language_denoise_layer (La  (None, 2, 3)              0         
 nguageDenoiseLayer)                                             
                                                                 
 listener (Functional)       (None, 1, 3)              95        
                                                                 
Total params: 215 (860.00 Byte)
Trainable params: 215 (860.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_______________________

In [7]:
inst = model.train(
    generator = gen,
    generations = 1,
    num_solutions = 50,
    num_parents_mating = 10,
)

c:\Users\nicho\PyProjects\CGAEL\.conda\Lib\site-packages\pygad\pygad.py:909: UserWarning: The steady-state parent (sss) selection operator is used despite that no parents are kept in the next generation.
  warnings.warn("The steady-state parent (sss) selection operator is used despite that no parents are kept in the next generation.")


Completed Generation: 1
Generating new batch...
Identifying best...


In [8]:
swatches = [Swatch.BLACK, Swatch.RED, Swatch.GREEN, Swatch.YELLOW, Swatch.BLUE, Swatch.MAGENTA, Swatch.CYAN, Swatch.WHITE]
samples = [sample_swatch(x) for x in swatches]
print(samples)

[[0.0, 0.0, 0.0], [0.0, 0.0, 1.0], [0.0, 1.0, 0.0], [0.0, 1.0, 1.0], [1.0, 0.0, 0.0], [1.0, 0.0, 1.0], [1.0, 1.0, 0.0], [1.0, 1.0, 1.0]]


In [14]:
KEY_SWATCH = "swatch"
KEY_IN = "input"
KEY_OUT = "output"
KEY_TEXT = "text"

swatches = [Swatch.BLACK, Swatch.RED, Swatch.GREEN, Swatch.YELLOW, Swatch.BLUE, Swatch.MAGENTA, Swatch.CYAN, Swatch.WHITE]
samples = np.array([[sample_swatch(x)] for x in swatches])
lang, out = model.model(samples)
text = [ts.decode(x) for x in lang]

d = {x:[] for x in [KEY_SWATCH, KEY_IN, KEY_OUT, KEY_TEXT]}
for s, t, i, o in zip(swatches, text, samples, out):
    d[KEY_SWATCH].append(s)
    d[KEY_TEXT].append(t)
    d[KEY_IN].append(i)
    d[KEY_OUT].append(o)
df = pd.DataFrame(data=d)
display(df)

['TAA HA', 'CC THC', 'C TA', 'CCT THC', 'T AA', 'CC TA', 'C TA', 'CCT TA']